In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [44]:
df = pd.read_csv('../data_clean/merged_total.csv')
df.shape

(250666, 19)

### Embedding

In [40]:
import openai
from dotenv import load_dotenv

In [41]:
load_dotenv("../.env")
openai.api_key = os.getenv('OPENAI_API_KEY')

embedding_model = "text-embedding-ada-002"

cat_cols = df.select_dtypes(include=['object']).columns
cat_cols = cat_cols[(cat_cols != 'visit_bounce') & (cat_cols != 'contact_contactpersoon_id') & (cat_cols != 'account_account_id')]

cat_cols

Index(['afspraak_keyphrases', 'account_keyphrases', 'campagne_keyphrases',
       'sessie_keyphrases', 'visit_keyphrases', 'mailing_keyphrases'],
      dtype='object')

In [42]:
def get_embedding(text):
    response = openai.Embedding.create(
    input=text,
    model=embedding_model
    )   
    return response['data'][0]['embedding']


def embed_col(df, col):
    unique_col = df[col].unique().tolist()
    dict_temp = {}

    for i in unique_col:
        dict_temp[i] = get_embedding(i)
    
    df[col+'_embed'] = df[col].map(dict_temp)
    df.drop(columns=[col], inplace=True)

    print(f'Column {col} embedded\nDataframe shape: {df.shape}')
    
    return df

In [45]:
df['mailing_keyphrases'].unique()

array(['unknown',
       'JO-Stamgasten 2023, Uitnodiging Stamgasten 23 mei 2023',
       'NW-Nieuwjaarsreceptie Vlaamse Ardennen - Leiestreek 2023, Nieuwjaarsreceptie Vlaamse Ardennen-Leiestreek | Voka Oost-Vlaanderen',
       'nieuwsbrief-09052023, Nieuws van Voka Oost-Vlaanderen, rechtstreeks in je inbox',
       'NW-Nieuwjaarsreceptie 2023 Gent, Nieuwjaarsreceptie Gent | Voka Oost-Vlaanderen',
       'NW-Community event Groei-Grootmeesters, Grootmeesters van de groei - inspirerend community event',
       '2023 04 24-Expert-Groeien door overname 2023, Groeien door overname',
       'NW-Nieuwjaarsreceptie VLAR 2023 Reminder, Last call | Nieuwjaarsreceptie Vlaamse-Ardennen & Leiestreek',
       'NW-Nieuwjaarsreceptie 2023 gastsprekers VLAR, Nieuwjaarsreceptie Vlaamse Ardennen & Leiestreek | Voka Oost-Vlaanderen',
       "NW-Praktische info: Connect Gent, Praktische info: Voka's nieuwjaarsreceptie in Gent",
       'NW/AO-Quick Refresh-Arbeidsdeal, Quick Refresh: Arbeidsdeal',
       '

### CLUSTERING

In [6]:
from sklearn.cluster import DBSCAN

In [ ]:
# reduce the value from each embedded column to a single value
def reduce_embedding(embedded):
    return np.mean(embedded)

df['campagne_naam_embedded'] = df['campagne_naam_embedded'].apply(lambda x: reduce_embedding(x))
df['visit_ip_embedded'] = df['visit_ip_embedded'].apply(lambda x: reduce_embedding(x))
df['afspraak_keyphrases_embedded'] = df['afspraak_keyphrases_embedded'].apply(lambda x: reduce_embedding(x))
df['mailing_name_embedded'] = df['mailing_name_embedded'].apply(lambda x: reduce_embedding(x))
df['mailing_onderwerp_embedded'] = df['mailing_onderwerp_embedded'].apply(lambda x: reduce_embedding(x))

df.shape

In [ ]:
dbscan = DBSCAN(eps=0.5, min_samples=5, ).fit(df)

In [ ]:
labels_db = dbscan.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels_db)) - (1 if -1 in labels_db else 0)
n_noise_ = list(labels_db).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

Estimated number of clusters: 60
Estimated number of noise points: 9604


In [ ]:
dbscan.get_params(deep=True)

{'algorithm': 'auto',
 'eps': 0.5,
 'leaf_size': 30,
 'metric': 'euclidean',
 'metric_params': None,
 'min_samples': 5,
 'n_jobs': None,
 'p': None}